In [134]:
import pandas as pd
import numpy as np
import ast
import regex as re
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, KFold

import matplotlib.pyplot as plt

In [135]:
# Set the display options to show more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [136]:
df = pd.read_csv('./data/gurgaon_properties_post_feature_selection_v2.csv').drop(columns=['store room', 'floor_category', 'balcony' ])
df.head(2)

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,flat,sector 36,0.82,3.0,2.0,New Property,850.0,0.0,0.0,Low
1,flat,sector 89,0.95,2.0,2.0,New Property,1226.0,1.0,0.0,Low


 Numeric ---> 'price', 'bedRoom', 'bathroom', 'built_up_area', 'servant room' 

 Orinal ---> 'furnishing_type', 'luxury_category'
 
 OneHot ---> 'property_type', 'sector', 'agePossession'

 'furnishing_type'
 1. 'unnfurnished' ---> 0
 2. 'semifusrnished ---> 1
 3. 'furnished' ---> 2

In [137]:
df['agePossession'] = df['agePossession'].replace({
    'Relatively New' : 'new',
    'Moderately Old' : 'old',
    'New Property': 'new',
    'Old Property' : 'old',
    'Under Construction' : 'undercons'
})

In [138]:
df['luxury_category'] = df['luxury_category'].replace({
    'Low' : 0,
    'Medium' : 1,
    'High': 2,
})

In [139]:
df['property_type'] = df['property_type'].replace({
    'flat' : 0,
    'house' : 1
})

In [140]:
newdf = pd.get_dummies(df, columns=['sector', 'agePossession'], drop_first=True)
newdf.head(2)

,property_type,price,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,sector_sector 1,sector_sector 10,sector_sector 102,sector_sector 103,sector_sector 104,sector_sector 105,sector_sector 106,sector_sector 107,sector_sector 108,sector_sector 109,sector_sector 11,sector_sector 110,sector_sector 111,sector_sector 112,sector_sector 113,sector_sector 12,sector_sector 13,sector_sector 14,sector_sector 15,sector_sector 17,sector_sector 2,sector_sector 21,sector_sector 22,sector_sector 23,sector_sector 24,sector_sector 25,sector_sector 26,sector_sector 27,sector_sector 28,sector_sector 3,sector_sector 30,sector_sector 31,sector_sector 33,sector_sector 36,sector_sector 37,sector_sector 37d,sector_sector 38,sector_sector 39,sector_sector 4,sector_sector 40,sector_sector 41,sector_sector 43,sector_sector 45,sector_sector 46,sector_sector 47,sector_sector 48,sector_sector 49,sector_sector 5,sector_sector 50,sector_sector 51,sector_sector 52,sector_sector 53,sector_sector 54,sector_sector 55,sector_sector 56,sector_sector 57,sector_sector 58,sector_sector 59,sector_sector 6,sector_sector 60,sector_sector 61,sector_sector 62,sector_sector 63,sector_sector 63a,sector_sector 65,sector_sector 66,sector_sector 67,sector_sector 67a,sector_sector 68,sector_sector 69,sector_sector 7,sector_sector 70,sector_sector 70a,sector_sector 71,sector_sector 72,sector_sector 73,sector_sector 74,sector_sector 76,sector_sector 77,sector_sector 78,sector_sector 79,sector_sector 8,sector_sector 80,sector_sector 81,sector_sector 82,sector_sector 82a,sector_sector 83,sector_sector 84,sector_sector 85,sector_sector 86,sector_sector 88,sector_sector 88a,sector_sector 89,sector_sector 9,sector_sector 90,sector_sector 91,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sohna road,agePossession_old,agePossession_undercons
0,0,0.82,3.0,2.0,850.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0.95,2.0,2.0,1226.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [141]:
X = newdf.drop(columns=['price'])
y_transformed = np.log1p(newdf['price'])

In [142]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled.head()

,property_type,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,sector_sector 1,sector_sector 10,sector_sector 102,sector_sector 103,sector_sector 104,sector_sector 105,sector_sector 106,sector_sector 107,sector_sector 108,sector_sector 109,sector_sector 11,sector_sector 110,sector_sector 111,sector_sector 112,sector_sector 113,sector_sector 12,sector_sector 13,sector_sector 14,sector_sector 15,sector_sector 17,sector_sector 2,sector_sector 21,sector_sector 22,sector_sector 23,sector_sector 24,sector_sector 25,sector_sector 26,sector_sector 27,sector_sector 28,sector_sector 3,sector_sector 30,sector_sector 31,sector_sector 33,sector_sector 36,sector_sector 37,sector_sector 37d,sector_sector 38,sector_sector 39,sector_sector 4,sector_sector 40,sector_sector 41,sector_sector 43,sector_sector 45,sector_sector 46,sector_sector 47,sector_sector 48,sector_sector 49,sector_sector 5,sector_sector 50,sector_sector 51,sector_sector 52,sector_sector 53,sector_sector 54,sector_sector 55,sector_sector 56,sector_sector 57,sector_sector 58,sector_sector 59,sector_sector 6,sector_sector 60,sector_sector 61,sector_sector 62,sector_sector 63,sector_sector 63a,sector_sector 65,sector_sector 66,sector_sector 67,sector_sector 67a,sector_sector 68,sector_sector 69,sector_sector 7,sector_sector 70,sector_sector 70a,sector_sector 71,sector_sector 72,sector_sector 73,sector_sector 74,sector_sector 76,sector_sector 77,sector_sector 78,sector_sector 79,sector_sector 8,sector_sector 80,sector_sector 81,sector_sector 82,sector_sector 82a,sector_sector 83,sector_sector 84,sector_sector 85,sector_sector 86,sector_sector 88,sector_sector 88a,sector_sector 89,sector_sector 9,sector_sector 90,sector_sector 91,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sohna road,agePossession_old,agePossession_undercons
0,-0.51718,-0.074329,-0.874300,-0.831662,-0.747968,-0.668281,-0.984642,-0.071348,-0.093805,-0.041123,-0.044424,-0.176186,-0.109357,-0.138615,-0.075228,-0.10396,-0.129928,-0.128804,-0.157474,-0.067248,-0.080708,-0.078923,-0.080708,-0.110667,-0.095319,-0.050386,-0.071348,-0.041123,-0.053119,-0.13215,-0.041123,-0.075228,-0.060591,-0.075228,-0.102569,-0.106691,-0.023729,-0.105334,-0.082455,-0.037535,-0.053119,-0.138615,12.670653,-0.121855,-0.134337,-0.053119,-0.05572,-0.113244,-0.037535,-0.05572,-0.128804,-0.047498,-0.05572,-0.067248,-0.125375,-0.113244,-0.077097,-0.13215,-0.044424,-0.062887,-0.062887,-0.085846,-0.053119,-0.123039,-0.080708,-0.044424,-0.047498,-0.05572,-0.065104,-0.108032,-0.067248,-0.062887,-0.065104,-0.15841,-0.108032,-0.115766,-0.077097,-0.092266,-0.163923,-0.09973,-0.118237,-0.124212,-0.087494,-0.090703,-0.029066,-0.071348,-0.065104,-0.087494,-0.067248,-0.147823,-0.044424,-0.037535,-0.15841,-0.108032,-0.073313,-0.139666,-0.118237,-0.177033,-0.133247,-0.029066,-0.078923,-0.12767,-0.078923,-0.160267,-0.069328,-0.170153,-0.050386,-0.125375,-0.109357,-0.219245,-0.602271,-0.290738
1,-0.51718,-0.877269,-0.874300,-0.522517,1.336956,-0.668281,-0.984642,-0.071348,-0.093805,-0.041123,-0.044424,-0.176186,-0.109357,-0.138615,-0.075228,-0.10396,-0.129928,-0.128804,-0.157474,-0.067248,-0.080708,-0.078923,-0.080708,-0.110667,-0.095319,-0.050386,-0.071348,-0.041123,-0.053119,-0.13215,-0.041123,-0.075228,-0.060591,-0.075228,-0.102569,-0.106691,-0.023729,-0.105334,-0.082455,-0.037535,-0.053119,-0.138615,-0.078923,-0.121855,-0.134337,-0.053119,-0.05572,-0.113244,-0.037535,-0.05572,-0.128804,-0.047498,-0.05572,-0.067248,-0.125375,-0.113244,-0.077097,-0.13215,-0.044424,-0.062887,-0.062887,-0.085846,-0.053119,-0.123039,-0.080708,-0.044424,-0.047498,-0.05572,-0.065104,-0.108032,-0.067248,-0.062887,-0.065104,-0.15841,-0.108032,-0.115766,-0.077097,-0.092266,-0.163923,-0.09973,-0.118237,-0.124212,-0.087494,-0.090703,-0.029066,-0.071348,-0.065104,-0.087494,-0.067248,-0.147823,-0.044424,-0.037535,-0.15841,-0.108032,-0.073313,-0.139666,-0.118237,-0.177033,-0.133247,-0.029066,-0.078

In [143]:
# use KFold iterative technique to be applied on LinearRegression
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(estimator=LinearRegression(), cv=kfold, n_jobs=-1, X=X_scaled, y=y_transformed, scoring='r2')

scores.mean(), scores.std()

(0.8512613057405425, 0.016992929105286228)

In [144]:
lr = LinearRegression()
lr.fit(X_scaled, y_transformed)

# create a dataframe showing contribution of each feature
print(lr.coef_.shape)
df_weight = pd.DataFrame(lr.coef_.reshape(1,lr.coef_.shape[0]), columns=X_scaled.columns)
df_weight


(112,)


,property_type,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,sector_sector 1,sector_sector 10,sector_sector 102,sector_sector 103,sector_sector 104,sector_sector 105,sector_sector 106,sector_sector 107,sector_sector 108,sector_sector 109,sector_sector 11,sector_sector 110,sector_sector 111,sector_sector 112,sector_sector 113,sector_sector 12,sector_sector 13,sector_sector 14,sector_sector 15,sector_sector 17,sector_sector 2,sector_sector 21,sector_sector 22,sector_sector 23,sector_sector 24,sector_sector 25,sector_sector 26,sector_sector 27,sector_sector 28,sector_sector 3,sector_sector 30,sector_sector 31,sector_sector 33,sector_sector 36,sector_sector 37,sector_sector 37d,sector_sector 38,sector_sector 39,sector_sector 4,sector_sector 40,sector_sector 41,sector_sector 43,sector_sector 45,sector_sector 46,sector_sector 47,sector_sector 48,sector_sector 49,sector_sector 5,sector_sector 50,sector_sector 51,sector_sector 52,sector_sector 53,sector_sector 54,sector_sector 55,sector_sector 56,sector_sector 57,sector_sector 58,sector_sector 59,sector_sector 6,sector_sector 60,sector_sector 61,sector_sector 62,sector_sector 63,sector_sector 63a,sector_sector 65,sector_sector 66,sector_sector 67,sector_sector 67a,sector_sector 68,sector_sector 69,sector_sector 7,sector_sector 70,sector_sector 70a,sector_sector 71,sector_sector 72,sector_sector 73,sector_sector 74,sector_sector 76,sector_sector 77,sector_sector 78,sector_sector 79,sector_sector 8,sector_sector 80,sector_sector 81,sector_sector 82,sector_sector 82a,sector_sector 83,sector_sector 84,sector_sector 85,sector_sector 86,sector_sector 88,sector_sector 88a,sector_sector 89,sector_sector 9,sector_sector 90,sector_sector 91,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sohna road,agePossession_old,agePossession_undercons
0,0.120165,0.054002,0.065119,0.210638,0.050946,0.008282,0.006186,0.009819,-0.022714,-0.005193,0.00524,0.027,-0.002906,0.001964,-0.019392,0.000572,-0.012987,0.016817,0.02615,-0.014966,0.008734,0.016229,0.030826,0.032171,-0.019458,-0.01225,0.028741,0.003291,0.013917,0.006722,-0.009418,0.035474,0.002196,0.017702,0.05756,0.073883,0.006742,0.041854,-0.012715,0.006276,0.022492,0.025957,0.002495,-0.011466,0.001143,0.012137,0.002577,-0.020775,0.007653,0.016786,0.060375,0.026957,0.015862,0.01286,0.054163,0.031743,-0.01723,0.071822,0.00021,0.004847,0.047764,0.045644,0.012616,0.011877,0.02523,0.023603,0.03182,-0.015749,0.023941,0.036719,0.04291,0.034841,0.023561,0.071046,0.05283,0.040919,0.008678,0.010935,0.018427,-0.007301,0.00552,0.017163,-0.003271,0.030231,-0.001439,0.022177,0.000257,0.000778,-0.004294,0.013167,-0.013463,0.000199,0.015684,0.000583,0.009752,-0.001083,-0.010296,0.008747,-0.003671,-0.00006,0.008601,0.002022,-0.013289,-0.012977,-0.00012,-0.028556,-0.003971,-0.025222,-0.010312,-0.029515,-0.0079,0.015381


In [145]:
coef_df = pd.DataFrame(lr.coef_.reshape(1,lr.coef_.shape[0]), columns=X_scaled.columns).stack().reset_index().drop(columns=['level_0']).rename(columns={'level_1':'feature', 0:'coef'})
coef_df

,feature,coef
0,property_type,0.120165
1,bedRoom,0.054002
2,bathroom,0.065119
3,built_up_area,0.210638
4,servant room,0.050946
5,furnishing_type,0.008282
6,luxury_category,0.006186
7,sector_gwal pahari,0.009819
8,sector_manesar,-0.022714
9,sector_sector 1,-0.005193


# Regression Analysis 
Check if the sample statistics also represents population parameters
Run regression analysis using stat module

In [146]:
import statsmodels.api as sm

# add contant to X_scaled
X_with_const = sm.add_constant(X_scaled)

# fit the modeel
model = sm.OLS(y_transformed, X_with_const).fit()

# obtain summary and analyse
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     196.7
Date:                Mon, 27 Nov 2023   Prob (F-statistic):               0.00
Time:                        20:37:48   Log-Likelihood:                 588.22
No. Observations:                3554   AIC:                            -950.4
Df Residuals:                    3441   BIC:                            -252.6
Df Model:                         112                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [147]:
# stdandardized_coef_bedRoom is the value when two transformations are applied
# 1. y is log transformed
# 2. X['bedRoom'] is StandardScaled
# To find correct impact we will need to reverse calculate un-standardized value

# Standardized coefficient for bedRoom
stdandardized_coef_bedRoom = coef_df[coef_df['feature'] == 'bedRoom']['coef'].values[0]
un_stdandardized_coef_bedRoom = stdandardized_coef_bedRoom * y_transformed.std()/X['bedRoom'].std()
# this un-standardized coefficient is actual impact of the value in price due to unit change in bedRoom value
print(un_stdandardized_coef_bedRoom)

np.expm1(un_stdandardized_coef_bedRoom)

0.024189791263896233


0.024484737693657145

In [148]:
# Standardized coefficient for built_up_area
stdandardized_coef_built_up_area = coef_df[coef_df['feature'] == 'built_up_area']['coef'].values[0]
un_stdandardized_coef_built_up_area = stdandardized_coef_built_up_area * y_transformed.std()/X['built_up_area'].std()
# this un-standardized coefficient is actual impact of the value in price due to unit change in built_up_area value
print(un_stdandardized_coef_built_up_area)

np.expm1(un_stdandardized_coef_built_up_area)

np.exp(un_stdandardized_coef_built_up_area) -1 

9.661685838204551e-05


9.66215259410319e-05